In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from transformers import BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch import FloatTensor
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

import tqdm

c:\Miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Глобальные константы, прописываем в отдельные преременные для удобства

RANDOM_SEED = 42
PATH_TO_RAW_DATA = os.path.join('data', 'raw', 'geo-reviews-dataset-2023.csv')
NUM_LABLES = 6
BATCH_SIZE = 40

In [3]:
# Загружаем датасэт

raw_data_set = pd.read_csv(PATH_TO_RAW_DATA)
print(f'Объем сырого датасэта: {len(raw_data_set)}')
raw_data_set.head(5)

Объем сырого датасэта: 500000


,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [4]:
lables = raw_data_set['rating'].unique() # возвращает объект типа np.ndarray
lables

array([3., 5., 1., 4., 2., 0.])

In [16]:
# Бьем на train и test, defaul shuffle = True

train_X, test_X, train_Y, test_Y = train_test_split(raw_data_set['text'], raw_data_set['rating'], train_size=0.3, test_size=0.1, random_state=(RANDOM_SEED + 1), stratify=raw_data_set['rating'])

In [18]:
# Токенизируем тексты

#tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny')
train_tokens = tokenizer(list(train_X), padding = True, truncation=True)
test_tokens = tokenizer(list(test_X), padding = True, truncation=True)

print(train_tokens.keys())
print(train_tokens['input_ids'][0])
print(tokenizer.decode(train_tokens['input_ids'][0]))

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[101, 459, 10286, 28395, 10286, 28398, 17424, 17127, 118, 492, 28399, 28400, 10286, 28396, 119, 453, 28403, 28404, 28414, 490, 10286, 20442, 28399, 24625, 28399, 10286, 119, 469, 19692, 17127, 28413, 490, 20442, 17424, 19692, 28401, 28400, 19692, 28401, 28413, 19692, 119, 465, 24625, 10286, 20442, 28413, 483, 490, 20442, 16948, 28396, 28405, 28399, 28404, 28413, 491, 10286, 28398, 17127, 16948, 16948, 28393, 20442, 10286, 28398, 17127, 28413, 19692, 117, 485, 10286, 28409, 19692, 28403, 28404, 28394, 19692, 17127, 17127, 28413, 19692, 483, 488, 19692, 489, 28409, 19692, 17127, 28414, 119, 457, 28405, 28402, 17424, 28400, 10286, 487, 17424, 28408, 19692, 28400, 14800, 20442, 17127, 28405, 28416, 477, 16948, 28396, 28405, 117, 485, 16948, 28404, 16948, 20442, 10286, 14800, 485, 20442, 10286, 28403, 17424, 28404, 486, 17424, 28408, 16948, 477, 491, 16948, 28398, 24625, 28413, 17106, 497, 28394, 19692, 28404, 483, 488, 17424, 284

In [19]:
# Создаем оболочку для хранения и передачи в модель наших данных

class CustomDataSet(Dataset):
    def __init__(self,
                 tokenized_text,
                 attention_mask,
                 lables):
        self.tokenized_text = torch.tensor(tokenized_text)
        self.attention_mask = torch.tensor(attention_mask)
        self.lables = torch.tensor(lables.to_numpy())

    def __len__(self):
        return len(self.lables)

    def __getitem__(self, idx):
        return self.tokenized_text[idx], self.attention_mask[idx], self.lables[idx]

In [ ]:
# Привер функции, которую можно переписать через CustomDataSet: collate_fn в DataSet из PyTorch

In [21]:
# Инициализируем два необходимых даталоадера

train_dataset = CustomDataSet(train_tokens['input_ids'], train_tokens['attention_mask'], train_Y)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)

test_dataset = CustomDataSet(test_tokens['input_ids'], test_tokens['attention_mask'], test_Y)
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=BATCH_SIZE)

# DataLoader основная функция это формирование батчей

In [22]:
# Загружаем нужную модель

model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=NUM_LABLES)
#model = AutoModelForSequenceClassification.from_pretrained('answerdotai/ModernBERT-base', num_labels=NUM_LABLES)

# model = AutoModel.from_pretrained("cointegrated/rubert-tiny") # Pre-trained model
optimizer = AdamW(model.parameters(), lr=1e-5) # Optimization function
loss_fn = torch.nn.CrossEntropyLoss() # Loss function

In [23]:
# Узнаем как выглядят токены:

test_id = train_tokens['input_ids'][0][1]
with torch.no_grad():
    test_embedding = model.get_input_embeddings()

test_embedding

In [24]:
len(test_embedding.weight[test_id])

In [20]:
# Выбираем доступное место для наших вычислений

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [25]:
num_epochs = 1
model.to(device) # Transfer model to GPU if available

In [ ]:
# ЦИКЛ ОБУЧЕНИЯ

train_losses = []
val_losses = []

train_losses_per_epoch = []
val_losses_per_epoch = []

for epoch in tqdm.tqdm(range(num_epochs)):
    # активируем обучение модели
    model.train()
    for batch in tqdm.tqdm(train_loader):

        tokenized_text, attention_mask, lables = batch
        lables = lables.type(torch.LongTensor)

        
        tokenized_text, attention_mask, lables = tokenized_text.to(device), attention_mask.to(device), lables.to(device)
        
        # Сбрасываем градиенты с прошлых шагов
        optimizer.zero_grad()
        
        # forward
        outputs = model(input_ids = tokenized_text, attention_mask = attention_mask)
        
        # Получаем логиты и передаем их в функцию потерь
        pred = outputs.logits
        # CrossEntropy в pytorch требуют именно logits, а не вероятности так как в начале CrossEntropy уже поставлен softmax
        loss = loss_fn(pred, lables)

        # Обратное распростронение
        loss.backward()
        
        # Шаг оптимизатора
        optimizer.step()

        # Сохраняем значение функции потерь для статистики 
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    #Замораживаем обучение
    model.eval()
    with torch.no_grad():
        for batch in test_loader:
            tokenized_text, attention_mask, lables = batch
            lables = lables.type(torch.LongTensor)
            tokenized_text, attention_mask, lables = tokenized_text.to(device), attention_mask.to(device), lables.to(device)
            
            # forward
            outputs = model(input_ids = tokenized_text, attention_mask = attention_mask)
            
            # Получаем логиты и передаем их в функцию потерь
            logits = outputs.logits
            loss = loss_fn(logits, lables)

            # Сохраняем значение функции потерь для статистики
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))
    val_losses_per_epoch = []
    train_losses_per_epoch = []